In [1]:
import numpy as np
import matplotlib as mpl
%matplotlib inline

import pandas as pd

import json
import glob
import os

In [2]:
data = {}

for path in glob.glob("../../data/results/lfr_params_ground_truth/*.json") + \
    glob.glob("../../data/results/lfr_params/**/*.json", recursive=True) + \
    glob.glob("../../data/results/all_real/*.json") + \
    glob.glob("../../data/results/all_real_seq/*.json") + \
    glob.glob("../../data/results/all_real_seq2/*.json") + \
    glob.glob("../../data/results/gossip_map/*.json") + \
    glob.glob("../../data/results/gossip_map2/*.json") + \
    glob.glob("../../data/results/louvain_baseline/*.json") + \
    glob.glob("../../data/results/infomap_baseline/*.json") + \
    glob.glob("../../data/results/weak_scaling/*.json") + \
    glob.glob("../../data/results/weak_scaling_small_quality/**/*.json", recursive=True):
  for typename, items in json.load(open(path)).items():
      if typename in data:
        for key, object_data in items.items():
          if key in data[typename]:
            data[typename][key].update(object_data)
          else:
            data[typename][key] = object_data
      else:
        data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

In [3]:
frames['algorithm_run']['runtime'].fillna((frames['algorithm_run']['done_ts'] - frames['algorithm_run']['start_ts']) / 1000000.0, inplace=True)

In [4]:
frames['program_run']['hosts'].fillna(1, inplace=True)
frames['program_run']['workers_per_host'].fillna(1, inplace=True)

In [9]:
data = frames['algorithm_run'].merge(frames['program_run'], left_on='program_run_id', right_index=True)
(data['runtime'] * data['hosts'] * data['workers_per_host']).sum() / 60 / 60 / 24 / 365

3.5908500805376624